In [1]:
import os
import dotenv

dotenv.load_dotenv()

True

In [3]:
env

{'ALLUSERSPROFILE': 'C:\\ProgramData',
 'APPCODE_VM_OPTIONS': 'C:\\swdtools\\jetbra\\vmoptions\\appcode.vmoptions',
 'APPDATA': 'C:\\Users\\akjam\\AppData\\Roaming',
 'CLION_VM_OPTIONS': 'C:\\swdtools\\jetbra\\vmoptions\\clion.vmoptions',
 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
 'COMPUTERNAME': 'JAMIE',
 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
 'DATAGRIP_VM_OPTIONS': 'C:\\swdtools\\jetbra\\vmoptions\\datagrip.vmoptions',
 'DATASPELL_VM_OPTIONS': 'C:\\swdtools\\jetbra\\vmoptions\\dataspell.vmoptions',
 'DEVECOSTUDIO_VM_OPTIONS': 'C:\\swdtools\\jetbra\\vmoptions\\devecostudio.vmoptions',
 'DPCPPROOT': 'C:\\Program Files (x86)\\Intel\\oneAPI\\dpcpp-ct\\2024.1',
 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
 'EFC_9852': '1',
 'GATEWAY_VM_OPTIONS': 'C:\\swdtools\\jetbra\\vmoptions\\gateway.vmoptions',
 'GOLAND_VM_OPTIONS': 'C:\\

In [4]:
# 导入sqlite3库
import sqlite3

# 连接到数据库
conn = sqlite3.connect('FlowerShop.db')
cursor = conn.cursor()

# 执行SQL命令来创建Flowers表
cursor.execute('''
        CREATE TABLE Flowers (
            ID INTEGER PRIMARY KEY, 
            Name TEXT NOT NULL, 
            Type TEXT NOT NULL, 
            Source TEXT NOT NULL, 
            PurchasePrice REAL, 
            SalePrice REAL,
            StockQuantity INTEGER, 
            SoldQuantity INTEGER, 
            ExpiryDate DATE,  
            Description TEXT, 
            EntryDate DATE DEFAULT CURRENT_DATE 
        );
    ''')

# 插入5种鲜花的数据
flowers = [
    ('Rose', 'Flower', 'France', 1.2, 2.5, 100, 10, '2023-12-31', 'A beautiful red rose'),
    ('Tulip', 'Flower', 'Netherlands', 0.8, 2.0, 150, 25, '2023-12-31', 'A colorful tulip'),
    ('Lily', 'Flower', 'China', 1.5, 3.0, 80, 5, '2023-12-31', 'An elegant white lily'),
    ('Daisy', 'Flower', 'USA', 0.7, 1.8, 120, 15, '2023-12-31', 'A cheerful daisy flower'),
    ('Orchid', 'Flower', 'Brazil', 2.0, 4.0, 50, 2, '2023-12-31', 'A delicate purple orchid')
]

for flower in flowers:
    cursor.execute('''
        INSERT INTO Flowers (Name, Type, Source, PurchasePrice, SalePrice, StockQuantity, SoldQuantity, ExpiryDate, Description) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
    ''', flower)

# 提交更改
conn.commit()

# 关闭数据库连接
conn.close()


OperationalError: table Flowers already exists

In [11]:
# 导入langchain的实用工具和相关的模块
from langchain.utilities import SQLDatabase
from langchain_google_genai import GoogleGenerativeAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.llms.tongyi import Tongyi

# os.environ["https_proxy"] = "http://127.0.0.1:7890"

# 连接到FlowerShop数据库（之前我们使用的是Chinook.db）
db = SQLDatabase.from_uri("sqlite:///FlowerShop.db")

# 创建OpenAI的低级语言模型（LLM）实例，这里我们设置温度为0，意味着模型输出会更加确定性
# llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0, verbose=True)
llm = Tongyi(dashscope_api_key=os.environ['DASHSCOPE_API_KEY'], temperature=0, verbose=True)

# 创建SQL数据库链实例，它允许我们使用LLM来查询SQL数据库
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

# 运行与鲜花运营相关的问题
response = db_chain.invoke("How many flower types are there？")
print(response)

response = db_chain.invoke("哪种鲜花的存货数量最少？")
print(response)

response = db_chain.invoke("平均销售价格是多少？")
print(response)

response = db_chain.invoke("从法国进口的鲜花有多少种？")
print(response)

response = db_chain.invoke("哪种鲜花的销售量最高？")
print(response)




> Entering new SQLDatabaseChain chain...
How many flower types are there？
SQLQuery:SELECT COUNT(DISTINCT "Type") FROM "Flowers"
SQLResult: [(1,)]
Answer:There is 1 flower type.
> Finished chain.
{'query': 'How many flower types are there？', 'result': 'There is 1 flower type.'}


> Entering new SQLDatabaseChain chain...
哪种鲜花的存货数量最少？
SQLQuery:SELECT "Name", "StockQuantity" FROM "Flowers" ORDER BY "StockQuantity" ASC LIMIT 1;
SQLResult: [('Orchid', 50)]
Answer:Orchid的存货数量最少，只有50个。
> Finished chain.
{'query': '哪种鲜花的存货数量最少？', 'result': 'Orchid的存货数量最少，只有50个。'}


> Entering new SQLDatabaseChain chain...
平均销售价格是多少？
SQLQuery:SELECT AVG("SalePrice") FROM "Flowers"
SQLResult: [(2.66,)]
Answer:平均销售价格是2.66。
> Finished chain.
{'query': '平均销售价格是多少？', 'result': '平均销售价格是2.66。'}


> Entering new SQLDatabaseChain chain...
从法国进口的鲜花有多少种？
SQLQuery:SELECT COUNT(DISTINCT "Name") FROM "Flowers" WHERE "Source" = 'France'
SQLResult: [(1,)]
Answer:从法国进口的鲜花有1种。
> Finished chain.
{'query': '从法国进口的鲜花有多少种？', 'resul

In [12]:
from langchain.utilities import SQLDatabase
from langchain_community.llms.tongyi import Tongyi
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

# 连接到FlowerShop数据库
db = SQLDatabase.from_uri("sqlite:///FlowerShop.db")
llm = Tongyi(dashscope_api_key=os.environ['DASHSCOPE_API_KEY'], temperature=0, verbose=True)

# 创建SQL Agent
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# 使用Agent执行SQL查询

questions = [
    "哪种鲜花的存货数量最少？",
    "平均销售价格是多少？",
]

for question in questions:
    response = agent_executor.run(question)
    print(response)




> Entering new SQL Agent Executor chain...
 我需要查询库存表中鲜花的数量，找出最少的数量。假设库存表名为'flowers_stock'，并且有字段'flower_name'代表鲜花名称，'quantity'代表数量。
Action: sql_db_query_checker
Action Input: SELECT flower_name, quantity FROM flowers_stock ORDER BY quantity ASC LIMIT 1The query appears to be correct. It selects the flower_name and quantity from the flowers_stock table, orders the results by quantity in ascending order, and limits the output to only one row. There are no mistakes related to the issues mentioned. Here's the final SQL query:

```sql
SELECT flower_name, quantity 
FROM flowers_stock 
ORDER BY quantity ASC 
LIMIT 1;
```查询看起来没有问题，现在可以执行它来获取答案。
Action: sql_db_query
Action Input: SELECT flower_name, quantity FROM flowers_stock ORDER BY quantity ASC LIMIT 1Error: (sqlite3.OperationalError) no such table: flowers_stock
[SQL: SELECT flower_name, quantity FROM flowers_stock ORDER BY quantity ASC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8) 查询失败是因为'flowers_stock'表不存在。我需要使用s